In [ ]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import regex

import nltk

nltk.data.path.append(r'C:\Users\roose\AppData\Roaming\nltk_data')

# # Baixar os pacotes necessários (se necessário)
nltk.download('punkt', download_dir=r'C:\Users\roose\AppData\Roaming\nltk_data')
nltk.download('stopwords', download_dir=r'C:\Users\roose\AppData\Roaming\nltk_data')
from wordcloud import WordCloud
from nltk.corpus import stopwords, words
from nltk.tokenize import WordPunctTokenizer, word_tokenize
from string import punctuation
from nltk.stem import WordNetLemmatizer

In [ ]:
names = []

caminho_atual = os.getcwd()

base = os.path.join(caminho_atual, 'Data')

folders = os.listdir(base)
data = []

# Ler os arquivos e juntar em um arquivo só
for folder in folders:
    files = os.listdir(os.path.join(base, folder))
    for file in files:
        try:
            if file == 'combine.csv':
                pass
            with open(os.path.join(base, folder, file), encoding='utf-8') as f:
                contents = " ".join(f.readlines())
                data.append([file.split(".")[0], folder, contents])
        except Exception as e:
            pass

df = pd.DataFrame(data, columns=['ID', 'Categoria', 'Conteudo'])

df
df['Categoria'].value_counts().plot.bar()

In [ ]:
unique = list(df.Conteudo.unique())


In [ ]:
unique = list(df.Conteudo.unique())
dic = dict(df)

uni = {}
i = 0
for k in range(len(list(dic['Conteudo']))):
    if dic['Conteudo'][k] in unique:
        uni[i] = [dic['Conteudo'][k], dic['Categoria'][k],dic['ID'][k]]
        unique.remove(dic['Conteudo'][k])
        i += 1


data = pd.DataFrame(uni).T
print(data.shape)
data.columns = ['Conteudo', 'Categoria','Id']
data.head()

In [ ]:
def make_wordcloud(words,title):
    cloud = WordCloud(width=1920, height=1080,max_font_size=200, max_words=300, background_color="white").generate(words)
    plt.figure(figsize=(20,20))
    plt.imshow(cloud, interpolation="gaussian")
    plt.axis("off") 
    plt.title(title, fontsize=60)
    plt.show()



In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()

stop = stopwords.words('english')

for punct in punctuation:
    stop.append(punct)

def filter_text(text, stop_words):
    word_tokens = WordPunctTokenizer().tokenize(text.lower())
    filtered_text = [regex.sub(u'\p{^Latin}', u'', w) for w in word_tokens if w.isalpha() and len(w) > 3]
    filtered_text = [wordnet_lemmatizer.lemmatize(w, pos="v") for w in filtered_text if not w in stop_words] 
    return " ".join(filtered_text)

In [ ]:
data["filtered_text"] = data.Conteudo.apply(lambda x : filter_text(x, stop)) 
data.head()

In [ ]:
all_text = " ".join(data[data.Categoria == "Crime"].filtered_text) 
make_wordcloud(all_text, "Crime")

In [ ]:
count = pd.DataFrame(all_text.split(), columns = ['words'])
top_10 = count[count['words'].isin(list(count.words.value_counts()[:10].index[:10]))]
plt.figure(figsize=(10,5))
sns.barplot(x = top_10.words.value_counts().index,
            y = top_10.words.value_counts(), palette = sns.color_palette("mako"))

In [ ]:
# Agora o seu código deve funcionar corretamente
# Configurar as stopwords e o lemmatizer
en_stopwords = stopwords.words('english')
word_lemmatizer = WordNetLemmatizer()

def clean_contents_w_stopwords(contents):
    tokenized_word = word_tokenize(str(contents).replace("\n", "").replace("\\", "").replace(">", "").strip())
    tokenized_word_w_stopwords = []
    for word in tokenized_word:
        word = regex.sub(u'\p{^Latin}', u'', word.lower())
        if len(word) > 3 and word.strip().lower() not in en_stopwords:
            word = word_lemmatizer.lemmatize(word)
            tokenized_word_w_stopwords.append(word)
    tokenized_word_w_stopwords = " ".join(tokenized_word_w_stopwords)
    return tokenized_word_w_stopwords

# Exemplo de uso
comment_words = df[df['ID'] == 14991]['Conteudo'].to_string().replace("\n", "").replace("\\", "").replace(">", "").strip()
print(comment_words)

tokenized_word_w_stopwords = clean_contents_w_stopwords(comment_words)
print(tokenized_word_w_stopwords)
